# 원인 불명의 환자에서 나온 미지의 종 찾기:  BLAST

### BLAST란,
'Basic Local Alignment Search Tool"의 줄임말로 **DNA, RNA 또는 아미노산 서열을 입력하면 데이터베이스에서 유사한 것을 찾아주는 알고리즘**입니다.
환자의 유전체파일(FASTQ) 에서 인간 유전체에 정렬되지 않는 서열들이 발견되었을 때, 이 서열들은 과연 무엇인지 BLAST에 넣어 어떤 종에서 유래한 것인지 알 수 있습니다. 

여기서는
1. BLAST를 사용하는 방법에 대해 알아보고
2. 바이오파이썬의 NCBIWWW모듈을 활용하여 BLAST를 수행해 봅니다.  

### 1. BLAST를 사용하는 방법
1. 웹브라우저에서 NCBI BLAST를 사용할 수 있습니다. 
https://blast.ncbi,nlm.nih.gov/Blast.cgi
2. 바이오파이썬으로 BLAST를 실행 할 수 있습니다. 

여기서는 바이오파이썬으로 실행하는 방법만 다루도록 하겠습니다. 

### 2. 바이오파이썬의 NCBIWWW모듈을 활용하여 BLAST를 수행

바이오파이썬의 NCBIWWW모듈에서 qblast() 메서드를 사용하여 BLAST을 진행합니다. 
1. 서열을 넣어 BLAST를 실행합니다. 
2. BLAST를 하여 나온 결과를 파싱합니다. 

#### NCBIWWW.qblast(BLAST프로그램명, BLAST데이터베이스, BLAST를 수행할 서열)
그래서 handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta")) 이럴 경우,
BLAST 프로그램의 종류는 blastn, 데이터베이스는 nt, 수행할 서열은 SeqIO로 FASTA파일을 읽은 SeqRecord 객체를 넣어주게 되는 것입니다. 

이렇게 넣어준 객체를 readlines()로 ㅇ릭은 후 출력하면 태그 형식의 문자열 결과들이 나오는데, XML포맷입니다. 

In [1]:
#1. NCBIWWW.qblast() implementation

from Bio.Blast import NCBIWWW #NCBIWWW.qblast() implementation할 때 필요 
from Bio import SeqIO #parsing할 때 필요 

record = SeqIO.read("buccal_swab.unmapped1.fasta", format="fasta")
handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))

result = handle.readlines()
for s in result:
    print(s)



<?xml version="1.0"?>

<!DOCTYPE BlastOutput PUBLIC "-//NCBI//NCBI BlastOutput/EN" "http://www.ncbi.nlm.nih.gov/dtd/NCBI_BlastOutput.dtd">

<BlastOutput>

  <BlastOutput_program>blastn</BlastOutput_program>

  <BlastOutput_version>BLASTN 2.10.0+</BlastOutput_version>

  <BlastOutput_reference>Stephen F. Altschul, Thomas L. Madden, Alejandro A. Sch&amp;auml;ffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J. Lipman (1997), &quot;Gapped BLAST and PSI-BLAST: a new generation of protein database search programs&quot;, Nucleic Acids Res. 25:3389-3402.</BlastOutput_reference>

  <BlastOutput_db>nt</BlastOutput_db>

  <BlastOutput_query-ID>Query_7365</BlastOutput_query-ID>

  <BlastOutput_query-def>buccal_swab.unmapped1</BlastOutput_query-def>

  <BlastOutput_query-len>88</BlastOutput_query-len>

  <BlastOutput_param>

    <Parameters>

      <Parameters_expect>10</Parameters_expect>

      <Parameters_sc-match>2</Parameters_sc-match>

      <Parameters_sc-mismatch>-3</Parameters_sc-m

#### NCBIXML.parse(NCBIWWW.qblast())
: 바이오파이썬에서 BLAST 실행 결과로 나온 XML 파일을 파싱하는 매서드입니다. 

In [ ]:
#2. parse BLAST result

from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO

#일단, BLAST를 수행하고
record = SeqIO.read("buccal_swab.unmapped1.fasta", format="fasta")
handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))

#parsing진행
blast_records = NCBIXML.parse(handle)
E_VALUE_THRESHOLD = 0.05
for record in blast_records:
    for alignment in record.alignments:
        for hsp in alignment.hsps: #hsp: 
            if hsp.expect < E_VALUE_THRESHOLD:
                print(alignment.title)
                print(alignment.length)
                print(hsp.expect)
                print(hsp.query[0:75])
                print(hsp.match[0:75])
                print(hsp.sbject[0:75])
            

### HSP
1. 우선 검색할 query 서열로 부터 3개의 단백질 혹은 11개의 염기로 이루어진 단어들과 T 이상의 점수를 가지는 조합을 만든다. 만들어진 조합들을 각각 서열 데이터베이스의 서열들과 비교한다.
2. 만약 각각의 단어 조합들과 같은 서열이 서열 database에서 발견이 되면 BLAST는 옆단어들로 유사성 검색을 확장시켜 나간다. 이때 gap은 허용하지 않는다.
3. 확장을 마친후 database 서열중 일정 값 이상의 HSP (High-scoring Segment Pairs) 를 가진 서열들을 추출하고 이때 중복되지 않는 각각의 HSP 들은 통계적인 test를 거쳐 연결한다.
4. BLAST는 quary 서열과 gap 없이 일정 값 이상의 HSP를 기록하지 못하는 서열들을 미리 제거한다. 그래서 FASTA에 비해 훨씬 비교속도가 빠르다. 하지만 두 서열이 특정 부분이 높은 일치성을 가지고 있지는 않지만 대부분의 서열에서 유사성을 가지고 있는 경우에 BLAST는 검색을 해 낼 수 가 없다.
5. Short repeat sequence 나 특정한 residue들이 많이 존재하는 서열들을 query 서열로 이용하였을 경우 많은 중요하지 않은 서열들이 결과로 나오게 된다. 이런 결과들을 피하기 위하여 BLAST는 filtering하는 기능을 기본적으로 가지고 있다. 결국 repeat sequence같은 것들은 검색하기 이전에 제거된다는 사실을 기억해야 한다. FASTA와 마찬가지로 BLAST도 단백질 서열을 위해 개발된 프로그램이다. 염기 서열의 검색이 가능하지만 sensitivity가 떨어지므로 염기 서열로 염기 데이터베이스를 검색해 진화적으로 떨어져 있는 서열을 찾고자 한다면 FASTA를 사용하는게 좋다.

출처: http://www.biohackers.net/wk/BLAST